In [8]:
import os
import io
import sys
import pandas as pd
import requests
import requests_cache
import pandas as pd

ELECTION_2016_CANDIDATES_URI = "https://results.aec.gov.au/20499/Website/Downloads/HouseMembersElectedDownload-20499.csv"
ELECTION_2016_RESULTSTPP_URI = "https://results.aec.gov.au/20499/Website/Downloads/HouseTcpByCandidateByVoteTypeDownload-20499.csv"

In [40]:
requests_cache.install_cache(".cache")

req_election2016_candidates = requests.get(ELECTION_2016_CANDIDATES_URI).text.split("\r\n", 1)[1] 
req_election2016_results = requests.get(ELECTION_2016_RESULTSTPP_URI).text.split("\r\n", 1)[1] 

election2016_candidates = pd.read_csv(io.StringIO(req_election2016_candidates))
election2016 = pd.read_csv(io.StringIO(req_election2016_results))

election2016.groupby(["DivisionID"])["TotalVotes"].sum()
# election2016
e = pd.merge(election2016.groupby(["DivisionID"])["TotalVotes"].sum(), election2016.loc[election2016["Elected"] == "Y"], left_on="DivisionID", right_on="DivisionID")

e["margin_votes"] = e["TotalVotes_y"] - (e["TotalVotes_x"] - e["TotalVotes_y"]) 
e["margin"] =  round(e["margin_votes"] / e["TotalVotes_x"] * 100, 2)
e["candidate"] = e["GivenNm"] + " " + e["Surname"]
e["electorate"] = e["DivisionNm"]
e["state"] = e["StateAb"]
e["party"] = e["PartyAb"]

e[[
    "electorate",
    "candidate",
    "state",
    "party",
    "margin_votes",
    "margin",
    "Swing"
]]

,electorate,candidate,state,party,margin_votes,margin,Swing
0,Canberra,Gai BRODTMANN,ACT,ALP,21774,16.92,0.95
1,Fenner,Andrew Keith LEIGH,ACT,ALP,34462,27.79,1.40
2,Banks,David COLEMAN,NSW,LP,2588,2.88,-1.36
3,Barton,Linda BURNEY,NSW,ALP,14560,16.60,3.91
4,Bennelong,John ALEXANDER,NSW,LP,17923,19.43,1.95
...,...,...,...,...,...,...,...
145,Flynn,Ken O'DOWD,QLD,LNP,1814,2.08,-5.49
146,Durack,Melissa PRICE,WA,LP,16957,22.11,-3.98
147,McMahon,Chris BOWEN,NSW,ALP,20846,24.23,7.48
148,Wright,Scott BUCHHOLZ,QLD,LNP,16984,19.23,-2.22
